In [1]:
import pandas as pd

In [2]:
from pathlib import Path

In [3]:
dPath = Path("../docs/dumps")

In [4]:
import pickle

In [5]:
with open(dPath / "train_data.pkl", 'rb') as filename:
    train_data = pickle.load(filename)

In [6]:
with open(dPath / "valid_data.pkl", 'rb') as filename:
    valid_data = pickle.load(filename)

In [7]:
X_train = train_data.drop("Detected", axis=1)

In [8]:
y_train = train_data.Detected

In [9]:
X_valid = valid_data.drop("Detected", axis=1)

In [10]:
y_valid = valid_data.Detected

In [11]:
with open(dPath / "rf_exp_01.pkl", 'rb') as filename:
    rf = pickle.load(filename)

In [12]:
names = X_train.columns

In [13]:
feat_importances = pd.Series(rf.feature_importances_, index=names)
feat_importances.sort_values(ascending=False,inplace=True)

In [24]:
names = list(feat_importances[:40].index)

In [26]:
X_train = X_train[names] 

In [28]:
X_valid = X_valid[names]

In [29]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
rf = RandomForestClassifier(n_estimators=20, max_features=0.5, oob_score=True, random_state=42, n_jobs=-1)

In [39]:
X_train.drop(['ppcumulativeNumberOfComments','ppcumulativeNumberOfCommentLines'], axis=1,inplace=True)

In [40]:
X_valid.drop(['ppcumulativeNumberOfComments','ppcumulativeNumberOfCommentLines'], axis=1,inplace=True)

In [41]:
%time rf.fit(X_train, y_train)

Wall time: 1min 8s


C:\Users\Alireza\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:523: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\Alireza\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:528: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=0.5,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=-1,
                       oob_score=True, random_state=42, verbose=0,
                       warm_start=False)

In [49]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [33]:
def print_score(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid), roc_auc_score(y_valid, m.predict(X_valid))]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [50]:
def conf_matr(m):
    y_pred = m.predict(X_valid)
    print(classification_report(y_valid, y_pred))

In [42]:
print_score(rf)

[0.9223711739789038, 0.7472887282209316, 0.55780409896931, 0.8466229764951095]


In [52]:
conf_matr(rf)

              precision    recall  f1-score   support

       False       0.47      0.18      0.26     16657
        True       0.78      0.93      0.85     50839

    accuracy                           0.75     67496
   macro avg       0.62      0.56      0.56     67496
weighted avg       0.70      0.75      0.70     67496



In [53]:
with open(dPath / "rf_exp_02.pkl", 'wb') as filename:
    pickle.dump(rf,filename)

In [44]:
rf.feature_importances_

array([0.11866474, 0.10651285, 0.06381462, 0.03697901, 0.04958234,
       0.03975827, 0.03795441, 0.02369213, 0.02221477, 0.02823176,
       0.01966977, 0.02497885, 0.0175287 , 0.02061433, 0.02468943,
       0.0160934 , 0.01671769, 0.02182595, 0.01756685, 0.01886641,
       0.0155541 , 0.0145643 , 0.01389753, 0.0158585 , 0.01277386,
       0.01163659, 0.01527346, 0.01510822, 0.01189274, 0.01940285,
       0.02037997, 0.01137902, 0.01513595, 0.0110467 , 0.01358644,
       0.02146322, 0.017602  , 0.01748825])

In [54]:
names = X_train.columns

In [55]:
feat_importances = pd.Series(rf.feature_importances_, index=names)
feat_importances.sort_values(ascending=False,inplace=True)

In [56]:
feat_importances

numExecuted                      0.118665
MutatorClass                     0.106513
numAssertInTC                    0.063815
numAssertInTM                    0.049582
ppavcc                           0.039758
cchalsteadCumulativeBugs         0.037954
numTestCover                     0.036979
ppnumberOfMethods                0.028232
ppmaintainabilityIndexNC         0.024979
mmhalsteadDifficulty             0.024689
cchalsteadCumulativeVolume       0.023692
ppRVF                            0.022215
mmnumberOfExpressions            0.021826
mmhalsteadEffort                 0.021463
ccmaintainabilityIndex           0.020614
mmhalsteadVocabulary             0.020380
ppnumberOfClasses                0.019670
ccmaintainabilityIndexNC         0.019403
mminstanceVariablesReferenced    0.018866
mmvariablesReferenced            0.017602
ccexternalMethodCalls            0.017567
ppfanout                         0.017529
ccunweightedClassSize            0.017488
ppmaintainabilityIndex           0

In [57]:
with open(dPath / "rf_exp_02_names.pkl", 'wb') as filename:
    pickle.dump(names,filename)